# **Семинар: Создание автоматизированного конвейера переобучения с помощью Apache Airflow**

## **Цель занятия**
Познакомиться с принципами оркестрации ML-конвейеров и научиться использовать Apache Airflow для управления задачами переобучения модели.

## **Задачи семинара**
- Понять архитектуру Airflow: DAG, операторы, сенсоры, scheduler.  
- Создать простой DAG для обучения и валидации модели.  
- Добавить логирование и уведомления.  
- Интегрировать DAG с Git и CI/CD.  

## **План**
1. Знакомство с Airflow и архитектурой DAG.  
2. Создание конвейера переобучения.  
3. Добавление логирования и уведомлений.  
4. Интеграция с внешними системами.  
5. Практическая работа: запуск DAG в локальной среде.


## **1. Знакомство с Apache Airflow**

**Airflow** — инструмент для оркестрации задач, который описывает процесс в виде *ориентированного ациклического графа* (DAG).  
Каждая вершина DAG — отдельная **task**, а рёбра определяют зависимости между ними.

Пример типичного ML-конвейера:

extract_data → preprocess → train_model → evaluate → deploy_model


Каждая задача выполняется независимо, с контролем времени, логированием и возможностью автоматического перезапуска при сбое.

**Airflow** идеально подходит для автоматизации переобучения моделей: можно запускать DAG по расписанию (например, раз в сутки), собирать новые данные и публиковать обновлённую модель.


## **2. Установка и настройка окружения**
Для локального запуска Airflow достаточно выполнить несколько команд.


In [ ]:
!pip install apache-airflow

Defaulting to user installation because normal site-packages is not writeable
  Using cached apache_airflow-2.5.1-py3-none-any.whl.metadata (121 kB)
  Using cached argcomplete-3.6.3-py3-none-any.whl.metadata (16 kB)
  Using cached attrs-25.4.0-py3-none-any.whl.metadata (10 kB)
  Using cached blinker-1.9.0-py3-none-any.whl.metadata (1.6 kB)
  Using cached cattrs-25.3.0-py3-none-any.whl.metadata (8.4 kB)
  Using cached colorlog-4.8.0-py2.py3-none-any.whl.metadata (9.8 kB)
  Using cached ConfigUpdater-3.2-py2.py3-none-any.whl.metadata (10 kB)
  Using cached connexion-3.3.0-py3-none-any.whl.metadata (12 kB)
  Using cached cron_descriptor-2.0.6-py3-none-any.whl.metadata (8.1 kB)
  Using cached croniter-6.0.0-py2.py3-none-any.whl.metadata (32 kB)
  Using cached deprecated-1.3.1-py2.py3-none-any.whl.metadata (5.9 kB)
  Using cached dill-0.4.0-py3-none-any.whl.metadata (10 kB)
  Using cached Flask-2.2.5-py3-none-any.whl.metadata (3.9 kB)
  Using cached Flask_AppBuilder-4.1.4-py3-none-any.whl.m

In [ ]:
# Установка Airflow
!pip install apache-airflow

# Инициализация базы данных
!airflow db init

# Создание администратора
!airflow users create \
  --username admin \
  --password admin \
  --firstname AI \
  --lastname Ops \
  --role Admin \
  --email admin@example.com

# Запуск сервисов (в отдельных терминалах)
# airflow webserver --port 8080
# airflow scheduler


## **3. Создание DAG для переобучения модели**

DAG (Directed Acyclic Graph) — это структура, описывающая зависимости между задачами.  
Каждая задача реализуется через **Operator** (например, `PythonOperator`, `BashOperator`, `EmailOperator`).

Ниже приведён минимальный пример конвейера обучения модели.


In [ ]:
# Файл: dags/retrain_pipeline.py

from airflow import DAG
from airflow.operators.python import PythonOperator
from datetime import datetime

def extract_data():
    print("Сбор данных из источника...")

def train_model():
    print("Обучение модели...")

def evaluate_model():
    print("Оценка качества модели...")

def deploy_model():
    print("Публикация новой версии модели...")

with DAG(
    "ml_retrain_pipeline",
    start_date=datetime(2025, 1, 1),
    schedule_interval="@daily",
    catchup=False,
    tags=["mlops", "retrain"]
) as dag:

    extract = PythonOperator(task_id="extract_data", python_callable=extract_data)
    train = PythonOperator(task_id="train_model", python_callable=train_model)
    evaluate = PythonOperator(task_id="evaluate_model", python_callable=evaluate_model)
    deploy = PythonOperator(task_id="deploy_model", python_callable=deploy_model)

    extract >> train >> evaluate >> deploy


## **4. Добавление логирования и уведомлений**

Airflow позволяет отправлять уведомления о статусе выполнения задач по email, Slack или Telegram.  
Ниже пример добавления задачи уведомления после успешного завершения пайплайна.


In [ ]:
# Добавим в тот же DAG задачу уведомления по email

from airflow.operators.email import EmailOperator

notify = EmailOperator(
    task_id="notify_success",
    to="team@mlops.local",
    subject="✅ ML Retrain Pipeline Completed",
    html_content="Новая модель успешно обучена и развернута.",
)

evaluate >> notify


## **5. Интеграция с Git и CI/CD**

Файлы DAG хранятся в папке `/opt/airflow/dags`.  
Можно автоматически обновлять их при пуше в основную ветку с помощью GitHub Actions.

Пример шага CI для синхронизации DAG на сервере Airflow:


In [ ]:
# .github/workflows/sync_dags.yml

name: Sync DAGs to Airflow

on:
  push:
    branches: [ main ]

jobs:
  deploy:
    runs-on: ubuntu-latest
    steps:
      - name: Checkout code
        uses: actions/checkout@v4

      - name: Sync DAGs to Airflow server
        run: rsync -avz dags/ user@airflow-server:/opt/airflow/dags/


## **6. Интерактивная часть**

Попробуйте выполнить следующие задания:

1. Запустите локальный Airflow и создайте свой DAG для переобучения модели.  
2. Добавьте задачу проверки метрик (например, чтобы метрика accuracy не ухудшилась).  
3. Настройте уведомление в Telegram или Slack при успешном завершении.  
4. Просмотрите журнал выполнения в Airflow UI (`http://localhost:8080`).  
5. Настройте расписание запуска (например, `schedule_interval="@hourly"`).  


## **Итоги семинара**

После выполнения всех шагов вы:

- Поняли принципы построения DAG в Airflow.  
- Создали собственный конвейер переобучения.  
- Добавили уведомления и автоматизацию CI/CD.  
- Научились запускать и отслеживать обновления модели по расписанию.  

Теперь вы готовы перейти к продвинутым сценариям — мониторинг задач, параметры DAG и интеграция с MLFlow или Docker.
